In [2]:
import pandas as pd
import numpy as np
import xarray as xr
import cdsapi 

# viz
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from cartopy.io import shapereader
import plotly.graph_objects as go

import glob
import sys
import os

# creds
sys.path.append('/Users/max/Deep_Sky')
from creds import CDS_UID, CDS_API_key

In [3]:

# point to local UNSEEN open dir to import retrieve and preprocess
sys.path.append('/Users/max/Deep_Sky/GitHub/UNSEEN-open-Deep-Sky')

import src.cdsretrieve as retrieve
import src.preprocess as preprocess

c = cdsapi.Client()

In [2]:
SEAS5 = preprocess.merge_SEAS5(folder ='../Data/drought/v3/SEAS5/', target_months = [5,6,7])
SEAS5

Lead time: 04
3
2


<xarray.Dataset>
Dimensions:    (latitude: 12, leadtime: 3, longitude: 21, number: 51, time: 129)
Coordinates:
  * number     (number) int64 0 1 2 3 4 5 6 7 8 9 ... 42 43 44 45 46 47 48 49 50
  * longitude  (longitude) float32 -111.1 -110.1 -109.1 ... -93.07 -92.07 -91.07
  * latitude   (latitude) float32 44.1 43.1 42.1 41.1 ... 36.1 35.1 34.1 33.1
  * time       (time) datetime64[ns] 1981-05-01 1981-06-01 ... 2023-07-01
  * leadtime   (leadtime) int64 2 3 4
Data variables:
    tprate     (leadtime, time, number, latitude, longitude) float32 dask.array<chunksize=(1, 3, 51, 12, 21), meta=np.ndarray>
    t2m        (leadtime, time, number, latitude, longitude) float32 dask.array<chunksize=(1, 3, 51, 12, 21), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-03-28 22:14:07 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [3]:
ERA5 = xr.open_mfdataset('../Data/drought/v3/ERA5/ERA5_????.nc', combine='by_coords')
ERA5

<xarray.Dataset>
Dimensions:    (latitude: 12, longitude: 21, time: 135)
Coordinates:
  * longitude  (longitude) float32 -111.1 -110.1 -109.1 ... -93.07 -92.07 -91.07
  * latitude   (latitude) float32 44.1 43.1 42.1 41.1 ... 36.1 35.1 34.1 33.1
  * time       (time) datetime64[ns] 1979-05-01 1979-06-01 ... 2023-07-01
Data variables:
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(3, 12, 21), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(3, 12, 21), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-03-28 23:08:27 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [4]:
def season_mean(ds, years, calendar='standard'):
    # Make a DataArray with the number of days in each month, size = len(time)
    month_length = ds.time.dt.days_in_month

    # Calculate the weights by grouping by 'time.season'
    weights = month_length.groupby('time.year') / month_length.groupby('time.year').sum()

    # Test that the sum of the weights for each season is 1.0
    np.testing.assert_allclose(weights.groupby('time.year').sum().values, np.ones(years))

    # Calculate the weighted average
    return (ds * weights).groupby('time.year').sum(dim='time', min_count = 3)

In [5]:
# Convert from m/s to total m for July (31 days)
SEAS5['tprate'] = SEAS5['tprate'] * 60 * 60 * 24

# Resample to yearly frequency and compute the mean
SEAS5_yearly = season_mean(SEAS5, len(pd.to_datetime(SEAS5.time.values).year.unique()))

# Update the attributes
SEAS5_yearly['tprate'].attrs = {'long_name': 'rainfall',
 'units': 'm/month',
 'standard_name': 'thickness_of_rainfall_amount'}

ERA5_yearly = season_mean(ERA5, len(pd.to_datetime(ERA5.time.values).year.unique()))

In [9]:
area_weights = np.cos(np.deg2rad(ERA5.latitude))

ERA5_events = (
    ERA5_yearly[['t2m','tp']].sel(  
        longitude = slice(-111.066, -90.631),    
        latitude = slice(44.898, 33.095)).          
    weighted(area_weights).
    mean(['longitude', 'latitude']) 
)

In [8]:
SEAS5_events = (
    SEAS5_yearly[['t2m','tprate']].sel(
        longitude = slice(-111.066, -90.631),   
        latitude = slice(44.898, 33.095)).     
    weighted(area_weights).
    mean(['longitude', 'latitude']))

In [10]:
SEAS5_events.to_dataframe().to_csv('../Data/drought/v3/SEAS5_events.csv')
ERA5_events.to_dataframe().to_csv('../Data/drought/v3/ERA5_events.csv')